In [1]:
from sage.all import * 

def unitary_divisors(n):
    return [x for x in divisors(n) if gcd(x,n/x)==1]

def kk(a,b):
    return gcd(a,b)**2/(a*b)

def plus(a,b):
    return a*b/(gcd(a,b)**2)

def mult(a,b):
    return gcd(a,b)

def compositionToDivisor(c):
    s = Composition(c).to_subset()
    if len(s)==0:
        return 1
    return prod([nth_prime(x) for x in s])

def rad(n):
    return prod(prime_divisors(n))

def divisorToComposition(d,n):
    if rad(d)==d:
        s = set([prime_pi(x) for x in prime_divisors(d)])
        return Composition(from_subset=(s,n))

def compositionKernel(c1,c2):
    a = compositionToDivisor(c1)
    b = compositionToDivisor(c2)
    return kk(a,b)

def primorial(n):
    return prod(primes(nth_prime(n)))


def pivoted_cholesky(M,max_rank=int(30)):
    import tensorflow_probability as tfp
    import tensorflow as tf
    import numpy as np
    piv_chol = (tfp.math.pivoted_cholesky(M,max_rank = max_rank, diag_rtol=float(0.001), name=None)).numpy()
    return piv_chol


# https://mathoverflow.net/questions/373475/a-geometric-approach-to-the-odd-perfect-number-problem
# embedding:
def jordanTotient(n,k=2):
    return n**2*prod([1-1/p**2 for p in prime_divisors(n)])

# embedding:
# https://mathoverflow.net/questions/373475/a-geometric-approach-to-the-odd-perfect-number-problem
def phi(C,axis=0):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np
    n = int(np.sum(list(C)))
    print("n = ",n)
    A = compositionToDivisor(C)
    print("A =",A)
    print("P_n = ",primorial(n-1))
    if axis == 1:
        S = csr_matrix((int(1),int(2**(n-1))), dtype=np.float64)
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))            
            S[0,x]         = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
    elif axis == 0:
        S = dok_matrix((int(2**(n-1)),int(1)), dtype=np.float64)
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))            
            S[x,0]         = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
    return S    

# embedding of all Compositions(n) in a dok_matrix
def phiN(n,maxN=8):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np
    if n > maxN:
        return None
    CC = Compositions(n)
    lC = 2**(n-1)
    S = dok_matrix((lC,lC),dtype=np.float64)
    #print(S.shape)
    c = 0
    #print(CC)
    for C in CC:
        #print(C)
        A = compositionToDivisor(list(C))
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))
            S[c,x] = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
        c+=1    
    return S

def fit_neighbors(n):
    X = phiN(n,maxN=8)
    CC = list(Compositions(n))
    from sklearn.neighbors import NearestNeighbors as NN
    neigh = NN()
    neigh = neigh.fit(X)
    return neigh,CC

def get_neighbors(neigh,CC,C,n_neighbors=5,reverse=True):
    v = phi(C,axis=1)
    dists, inds = (neigh.kneighbors(v, n_neighbors, return_distance=True))
    ll = []
    for i in inds[0]:
        ll.append(CC[i])
    if reverse:
        return ll[-1::-1]
    else:    
        return ll    

neigh,CC = fit_neighbors(8)
ll = get_neighbors(neigh,CC,[2,1,1,2,1,1],n_neighbors=5)
print(ll)

n =  8
A = 23205
P_n =  30030
[[1, 2, 1, 2, 1, 1], [2, 2, 2, 1, 1], [3, 1, 2, 1, 1], [1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 2, 1, 1]]


/usr/lib/python3/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [7]:
import music21 as m21
score = m21.stream.Score()
tempo  = 150
tm = m21.tempo.MetronomeMark(number=tempo)
score.append(tm)

def getPadicDigits(q=2/3,p=2,prec=100):
    p = p
    RDigits = Qp(p=p,prec=prec,type="lattice-cap",print_mode="bars",print_sep=",")
    R = RDigits
    digits = ([ int(x) for x in str(R(q)).replace(".","").split(",") if len(x)>0])
    #digits.reverse()
    return digits[0:prec]


vollist = [128//8*k for k in [3,4,5,6]]

def harmonic(t,y0,y1,periodendauer):
    T = periodendauer
    #X = matrix([[np.sin(np.pi/2+phi0),1],[np.sin(np.pi/2),1]])
    #print(X)
    A = (y0-y1)/2
    B = (y0+y1)/2
    #print(A,B)
    frequenz = 1.0/T
    return int(np.round(A*np.cos(2*np.pi*frequenz*t)+B,0))

def getHarmonicFromList(t,ll,T):
    periodendauer = T
    y0 = 0
    y1 = len(ll)-1
    return ll[harmonic(t,y0,y1,periodendauer)]

def convertPitchNrToNote(pitchNr):
    Z12ToPitch = dict(zip(range(12),["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]))
    octave = pitchNr // 12
    pitch = pitchNr % 12
    
    note = Z12ToPitch[pitch]    
    
    return note +  str(octave)

parts = []
scale = [60 + x for x in [0,2,4,5,7,9,11]]

drum = list(range(41,48+1))

class WarmPad(m21.instrument.Instrument):
    '''
    >>> p = instrument.Piano()
    >>> p.instrumentName
    'Piano'
    >>> p.midiProgram
    0
    '''

    def __init__(self):
        super().__init__()

        self.instrumentName = 'Warm Pad'
        self.instrumentAbbreviation = 'W.P.'
        self.midiProgram = 89

        self.lowestNote = m21.pitch.Pitch('A0')
        self.highestNote = m21.pitch.Pitch('C8')

        
def minkowskiSum(A,B):
    return ([a+b for a in A for b in B])

def k_Minkowski(k,A):
    if k < 0:
        return None
    
    if k == 1:
        return A
    else:
        return minkowskiSum(k_Minkowski(k-1,A),A)


def cyclicExtension(nn):
    k = len(nn)
    ll = [n for n in nn]
    for i in range(k):
        ll.extend([nn[(j+i)%k] for j in range(k)])
    return ll    

def applyPermutationToList(perm,ll):
    if len(perm)==len(ll) and sorted(list(set(perm)))==list(range(len(ll))):
        return [ll[perm[i]] for i in range(len(perm))]

def getBarNumber(duration):
    import numpy as np
    return int(np.floor(duration/4.0))

def getSmallestPeriodicRational(periodlength=1,prime=7):
    n = 2
    while True:
        Zn = Zmod(n)
        for x in range(1,n):
            if gcd(x,prime)==1 and gcd(n,prime)==1 and multiplicative_order(Zn(prime))==periodlength:
                return(-x/n)
        n+=1    

def cumSum(ll):
    return [sum(ll[0:i]) for i in range(len(ll)+1)]
        
piano = m21.instrument.Piano()
woodblock = m21.instrument.Woodblock()
taikodrum = m21.instrument.Taiko()
percussion = tomtom
warmpad = WarmPad()
marimba = m21.instrument.Marimba()
harp = m21.instrument.Harp()
string = m21.instrument.StringInstrument()

nrInst = 20
for k in range(nrInst):
    lh = m21.stream.Part()
    #lh.append(m21.instrument.Piano())
    #lh.append(WarmPad())
    if k%10 in [0]:
        lh.append(piano)
    elif k%10 in [1]:
        inst = piano #m21.instrument.AcousticBass()
        #inst.midiChannel = 9
        #inst.midiProgram = 53
        #inst.instrumentName = 'Orchestra Kit'
        #inst.instrumentAbbreviation = 'O.K.'
        print(inst)
   # 1 Standard Kit
   # 9 Room Kit
   # 17 Power Kit
   # 25 Electronic Kit
   # 26 TR-808 Kit
   # 33 Jazz Kit
   # 41 Brush Kit
   # 49 Orchestra Kit
   # 57 Sound FX Kit
   # 128 Percussion      
        lh.append(piano)
    elif k%10 in [2,3]:
        lh.append(taikodrum) #m21.instrument.Piano())
    elif k%10 in [4,5]:
        lh.append(harp) #m21.instrument.Harp())
    elif k%10 in [6,7]:
        lh.append(warmpad) #m21.instrument.Marimba())
    elif k%10 in [8,9]:
        lh.append(string) #WarmPad())
    parts.append(lh)


#shuffle(parts)    
    
prec = 256*20

import numpy as np

Npartition = 5
#op = OrderedPartitions(Npartition).list()[1:]

#shuffle(op)
changeAfterNSeconds = 5
last_tim = -changeAfterNSeconds
cnt = 0
nBars = 2**7
print(nBars)
if prec < nBars:
    print("......")
    
N = 5*3
possibleParts = [2,4,8]
#print(numbers)
cnt = 0
numbers = [getSmallestPeriodicRational(periodlength=d,prime=7) for d in divisors(N) if d > 1]
restsNumbers = [getSmallestPeriodicRational(periodlength=d,prime=2) for d in divisors(N) if d > 1]
print("restsNumbers = ",restsNumbers)


lB = floor(log(nBars)/log(2))+1

print(nrInst)
onOffInsts = [range(x*nrInst//lB,(x+1)*nrInst//lB) for x in range(lB)]
instsToPlay = []
for b in range(2**lB):
    digits2 = Integer(b).digits(2,padto=lB)
    print(digits2)
    playInst = []
    for x in range(nrInst):
        if any([x in onOffInsts[i] for i in range(lB) if digits2[i]==1]):
            pI = 1
        else:
            pI = 0
        playInst.append(pI)    
    print(playInst) 
    instsToPlay.append(playInst)


cnt = 0
x = [3,1,4]
neigh,CC = fit_neighbors(8)
LL = []
for i in range(len(parts)):
    digits = getPadicDigits(q=numbers[i%len(numbers)],p=7,prec=prec)
    rests = getPadicDigits(q=restsNumbers[i%len(restsNumbers)],p=2,prec=prec)
    durs = getPadicDigits(q=numbers[1],p=2,prec=prec)
    nPartitions = getPadicDigits(q=numbers[i%len(numbers)],p=3,prec=prec)
    m = len(digits)
    #print(i,numbers[i])
    #print(digits)
    print(i,nPartitions)
    #print(durs)
    sumDur = 0
    sumNN = 0
    #cnt = 0
    for bn in range(1,2**lB+1):
        time_in_minutes = bn*1/tempo*1.0
        tim = int(np.round(time_in_minutes*60/changeAfterNSeconds,0)*changeAfterNSeconds)
        
        playInst = instsToPlay[bn-1]
        volume = getHarmonicFromList(sum(playInst),vollist,T=2*len(parts))
        print("volume",volume,tim,time_in_minutes,bn,cnt)
        if tim-last_tim==changeAfterNSeconds and tim in [changeAfterNSeconds*x for x in cumSum([1,2,4,8,16,8,4])]:
            if len(LL)==0:
                ll = get_neighbors(neigh,CC,x,n_neighbors=20,reverse=True)
                LL.extend(ll)
            #op = OrderedPartitions(nPart).list()[:0:-1]
            #shuffle(op)
            xdurs = LL.pop(0)
            #shuffle(xdurs)
            print(xdurs,x,LL)
            x = xdurs
            cnt += 1
            nPart = np.sum(list(xdurs))
        last_tim = tim
        j = 0
        for digit in digits[sumNN:(sumNN+len(xdurs))]:
            #isRest = (rests[j]==1)
            pitchNr = scale[digit]
            D = float(xdurs[j%len(xdurs)]*4.0/nPart)   
            j+=1
            sumDur += D#print(bn,bnDigits)                 
            
            if playInst[i]==0:
                n0 = m21.note.Rest()
            else:    
                if i%10 in [-1]:# violine
                    n0 = m21.note.Note(convertPitchNrToNote(pitchNr-24))
                else:
                    n0 = m21.note.Note(convertPitchNrToNote(pitchNr-(i%2)*24))
                #n0 = m21.note.Note(convertPitchNrToNote(pitchNr-24))    
                n0.volume.velocity = volume        
            duration = m21.duration.Duration(D)
            n0.duration = duration #getDurationInQuarterByEmphasis(emph,factor=factor)
            parts[i].append(n0)
        sumNN+=len(xdurs)

for part in parts:
    score.append(part)
            
score.insert(0,m21.metadata.Metadata())    
title = "Hills vapor"
score.metadata.title = title

score.metadata.composer = 'musescore1983'
score.write("midi", fp=("_".join(title.split(" ")))+".mid")     
score.write("musicxml", fp=("_".join(title.split(" ")))+".musicxml")     
    

Piano
Piano
128
restsNumbers =  [-0.14285714285714285, -0.03225806451612903, -0.006622516556291391]
20
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 1,

0 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

volume 48 25 0.43333333333333335 65 7
volume 48 25 0.44 66 7
volume 48 25 0.44666666666666666 67 7
volume 64 25 0.4533333333333333 68 7
volume 48 30 0.46 69 7
volume 64 30 0.4666666666666667 70 7
volume 64 30 0.47333333333333333 71 7
volume 64 30 0.48 72 7
volume 48 30 0.4866666666666667 73 7
volume 64 30 0.49333333333333335 74 7
volume 64 30 0.5 75 7
volume 80 30 0.5066666666666667 76 7
volume 64 30 0.5133333333333333 77 7
volume 64 30 0.52 78 7
volume 80 30 0.5266666666666666 79 7
volume 80 30 0.5333333333333333 80 7
volume 48 30 0.54 81 7
volume 64 35 0.5466666666666666 82 7
[2, 1, 5] [1, 2, 1, 1, 3] [[3, 1, 3, 1], [2, 2, 4], [1, 2, 5], [3, 1, 2, 2], [3, 1, 1, 3], [1, 3, 4], [3, 5], [1, 1, 1, 1, 4], [4, 4], [2, 1, 1, 4], [1, 2, 1, 4], [3, 1, 4]]
volume 64 35 0.5533333333333333 83 8
volume 64 35 0.56 84 8
volume 64 35 0.5666666666666667 85 8
volume 64 35 0.5733333333333334 86 8
volume 64 35 0.58 87 8
volume 80 35 0.5866666666666667 88 8
volume 64 35 0.5933333333333334 89 8
volume 64 

volume 80 65 1.1133333333333333 167 12
volume 80 65 1.12 168 12
volume 64 70 1.1266666666666667 169 12
volume 80 70 1.1333333333333333 170 12
volume 80 70 1.14 171 12
volume 80 70 1.1466666666666667 172 12
volume 80 70 1.1533333333333333 173 12
volume 80 70 1.16 174 12
volume 80 70 1.1666666666666667 175 12
volume 96 70 1.1733333333333333 176 12
volume 64 70 1.18 177 12
volume 80 70 1.1866666666666668 178 12
volume 80 70 1.1933333333333334 179 12
volume 80 70 1.2 180 12
volume 80 70 1.2066666666666668 181 12
volume 80 75 1.2133333333333334 182 12
[3, 1, 1, 3] [3, 1, 2, 2] [[1, 3, 4], [3, 5], [1, 1, 1, 1, 4], [4, 4], [2, 1, 1, 4], [1, 2, 1, 4], [3, 1, 4]]
volume 80 75 1.22 183 13
volume 96 75 1.2266666666666666 184 13
volume 80 75 1.2333333333333334 185 13
volume 80 75 1.24 186 13
volume 80 75 1.2466666666666666 187 13
volume 96 75 1.2533333333333334 188 13
volume 80 75 1.26 189 13
volume 96 75 1.2666666666666666 190 13
volume 96 75 1.2733333333333334 191 13
volume 96 75 1.28 192 13
vol

4 [0, 0, 2, 2, 0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 1, 0, 0, 2, 2, 1, 0, 0, 1, 2, 2, 1, 1, 2, 0, 1, 1, 0, 2, 2, 2, 1, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 2, 2, 0, 0, 0, 2, 1, 0, 1, 2, 1, 0, 2, 0, 2, 1, 0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 1, 2, 0, 1, 0, 2, 2, 0, 0, 1, 0, 2, 2, 1, 2, 2, 2, 0, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 0, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 2, 1, 0, 2, 1, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 0, 0, 2, 1, 2, 2, 0, 2, 2, 1, 0, 2, 2, 0, 1, 2, 0, 2, 0, 0, 0, 1, 2, 0, 1, 0, 0, 1, 2, 1, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 0, 2, 1, 2, 0, 1, 2, 0, 1, 2, 0, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 2, 1, 1, 0, 0, 0, 1, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 1, 2, 0, 1, 1, 0, 1, 1, 0, 1, 1, 2, 0, 0, 2, 0, 2, 0, 1, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 1, 0, 1, 2, 2, 0, 0, 0, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 2, 0, 1, 0, 2, 2, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 2

...

volume 80 85 1.4533333333333334 218 61
volume 80 90 1.46 219 61
volume 96 90 1.4666666666666666 220 61
volume 80 90 1.4733333333333334 221 61
volume 80 90 1.48 222 61
volume 96 90 1.4866666666666666 223 61
volume 96 90 1.4933333333333334 224 61
volume 64 90 1.5 225 61
volume 80 90 1.5066666666666666 226 61
volume 80 90 1.5133333333333334 227 61
volume 80 90 1.52 228 61
volume 80 90 1.5266666666666666 229 61
volume 80 90 1.5333333333333334 230 61
volume 80 90 1.54 231 61
volume 96 95 1.5466666666666666 232 61
volume 80 95 1.5533333333333332 233 61
volume 80 95 1.56 234 61
volume 80 95 1.5666666666666667 235 61
volume 96 95 1.5733333333333333 236 61
volume 80 95 1.58 237 61
volume 96 95 1.5866666666666667 238 61
volume 96 95 1.5933333333333333 239 61
volume 96 95 1.6 240 61
volume 80 95 1.6066666666666667 241 61
volume 80 95 1.6133333333333333 242 61
volume 80 95 1.62 243 61
volume 96 100 1.6266666666666667 244 61
volume 80 100 1.6333333333333333 245 61
volume 80 100 1.64 246 61
volume 9

volume 48 5 0.04666666666666667 7 65
[1, 2, 1, 2, 2] [1, 1, 2, 4] [[1, 2, 1, 1, 3], [2, 1, 5], [3, 1, 3, 1], [2, 2, 4], [1, 2, 5], [3, 1, 2, 2], [3, 1, 1, 3], [1, 3, 4], [3, 5], [1, 1, 1, 1, 4], [4, 4], [2, 1, 1, 4], [1, 2, 1, 4], [3, 1, 4]]
volume 64 5 0.05333333333333334 8 66
volume 48 5 0.06 9 66
volume 48 5 0.06666666666666667 10 66
volume 64 5 0.07333333333333333 11 66
volume 64 5 0.08 12 66
volume 48 5 0.08666666666666667 13 66
volume 64 5 0.09333333333333334 14 66
volume 64 5 0.1 15 66
volume 80 5 0.10666666666666667 16 66
volume 48 5 0.11333333333333333 17 66
volume 48 5 0.12 18 66
volume 48 10 0.12666666666666668 19 66
volume 64 10 0.13333333333333333 20 66
volume 48 10 0.14 21 66
volume 64 10 0.14666666666666667 22 66
volume 64 10 0.15333333333333332 23 66
volume 64 10 0.16 24 66
volume 48 10 0.16666666666666666 25 66
volume 64 10 0.17333333333333334 26 66
volume 64 10 0.18 27 66
volume 80 10 0.18666666666666668 28 66
volume 64 10 0.19333333333333333 29 66
volume 64 10 0.2 30

volume 64 65 1.08 162 72
volume 64 65 1.0866666666666667 163 72
volume 80 65 1.0933333333333333 164 72
volume 64 65 1.1 165 72
volume 80 65 1.1066666666666667 166 72
volume 80 65 1.1133333333333333 167 72
volume 80 65 1.12 168 72
volume 64 70 1.1266666666666667 169 72
volume 80 70 1.1333333333333333 170 72
volume 80 70 1.14 171 72
volume 80 70 1.1466666666666667 172 72
volume 80 70 1.1533333333333333 173 72
volume 80 70 1.16 174 72
volume 80 70 1.1666666666666667 175 72
volume 96 70 1.1733333333333333 176 72
volume 64 70 1.18 177 72
volume 80 70 1.1866666666666668 178 72
volume 80 70 1.1933333333333334 179 72
volume 80 70 1.2 180 72
volume 80 70 1.2066666666666668 181 72
volume 80 75 1.2133333333333334 182 72
[3, 1, 1, 3] [3, 1, 2, 2] [[1, 3, 4], [3, 5], [1, 1, 1, 1, 4], [4, 4], [2, 1, 1, 4], [1, 2, 1, 4], [3, 1, 4]]
volume 80 75 1.22 183 73
volume 96 75 1.2266666666666666 184 73
volume 80 75 1.2333333333333334 185 73
volume 80 75 1.24 186 73
volume 80 75 1.2466666666666666 187 73
volu

19 [0, 0, 2, 2, 0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 1, 0, 0, 2, 2, 1, 0, 0, 1, 2, 2, 1, 1, 2, 0, 1, 1, 0, 2, 2, 2, 1, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 2, 2, 0, 0, 0, 2, 1, 0, 1, 2, 1, 0, 2, 0, 2, 1, 0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 1, 2, 0, 1, 0, 2, 2, 0, 0, 1, 0, 2, 2, 1, 2, 2, 2, 0, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 0, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 2, 1, 0, 2, 1, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 0, 0, 2, 1, 2, 2, 0, 2, 2, 1, 0, 2, 2, 0, 1, 2, 0, 2, 0, 0, 0, 1, 2, 0, 1, 0, 0, 1, 2, 1, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 0, 2, 1, 2, 0, 1, 2, 0, 1, 2, 0, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 2, 1, 1, 0, 0, 0, 1, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 1, 2, 0, 1, 1, 0, 1, 1, 0, 1, 1, 2, 0, 0, 2, 0, 2, 0, 1, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 1, 0, 1, 2, 2, 0, 0, 0, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 2, 0, 1, 0, 2, 2, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 

'/home/orges/notebooks/algorithmic_python_music/Hills_vapor.xml'

In [25]:
for x in numbers:
    #print(x)
    print("".join([str(xx) for xx in getPadicDigits(q=x,p=7,prec=57)]))

515151515151515151515151515151515151515151515151515151515
053053053053053053053053053053053053053053053053053053053
346012346012346012346012346012346012346012346012346012346
